In [1]:
import urllib.request as request
import json
import boto3
import pandas as pd

In [6]:
#read the timestamd data to combine into a filename to load 
s3_read = boto3.resource('s3')
s3_bucket = boto3.client('s3')
bucket_response = s3_bucket.list_buckets()

In [7]:
bucket_response

{'ResponseMetadata': {'RequestId': '1E1AE2EEFF70F5E3',
  'HostId': 'JPZWcwFo1UbMlmm2wHKx3wBH9OUqd6zGbqly31Hfa3Bf/2TwoZN7Ip56AN0O25zinCywTT0DRpk=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'JPZWcwFo1UbMlmm2wHKx3wBH9OUqd6zGbqly31Hfa3Bf/2TwoZN7Ip56AN0O25zinCywTT0DRpk=',
   'x-amz-request-id': '1E1AE2EEFF70F5E3',
   'date': 'Thu, 06 Aug 2020 05:45:26 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [],
 'Owner': {'DisplayName': 'lab+LabServices-Prod-2757',
  'ID': '3feeb722be952f7d484a6aa8a8afbd23aeeaae3389c39bbe4cebe87c543dbf59'}}

In [ ]:
bucketname = 'capstone-team-uk-data-dump-bucket'
itemname = 'raw-zone/timestamp_file_hospital.txt'
obj = s3_read.Object(bucketname, itemname)
body_timestamp = obj.get()['Body'].read()

In [67]:
hospital_filename = 'hospital-data-'+body_timestamp.decode()+'.json'

In [46]:
#load from s3 raw zone
s3_read = boto3.resource('s3')
bucketname = 'capstone-team-uk-data-dump-bucket'
itemname = 'raw-zone/'+hospital_filename
obj = s3_read.Object(bucketname, itemname)
body = obj.get()['Body'].read()

In [47]:
data = json.loads(body)

In [50]:
json_simple = data['features']

In [53]:
new_list_of_dicts = [
    x['attributes'] for x in json_simple
]

In [54]:
data_f = pd.DataFrame(new_list_of_dicts)

In [55]:
data_f.index.name = 'id'

In [57]:
output_csv = pd.DataFrame.to_csv(data_f)

In [59]:
output_csv = data_f.to_csv('hospital-data.csv')

In [69]:
timestamp = body_timestamp.decode()

In [72]:
s3_upload = boto3.client('s3')
with open('hospital-data.csv', "rb") as f:
    s3_upload.upload_fileobj(f, "capstone-team-uk-data-dump-bucket", "hospital-data/hospital-data"+timestamp+".csv")